In [46]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *

import scipy.stats as stats
from scipy.stats import wilcoxon
from scipy.stats import shapiro
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [61]:
key = 'Infected'
var_list = ['up1', 'up2', 'down1', 'down2']
cities = ['강원', '경기', '경남', '경북', '광주', '대구', 
          '대전', '부산', '서울', '세종', '울산', '인천',
          '전남', '전북', '제주', '충남', '충북']
models = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
          'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
          'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

In [62]:
dic_best_model = {}
for var in var_list:
    dic_best_model_sub = {}
    for city in cities:
        path = f'Data/Predicted/Stricgency/{key}/{var}/{city}/best*.csv'
        file = glob.glob(path)
        df = pd.read_csv(file[0], encoding = 'cp949')
        df.columns = [['Date', 'Actual_data', 'Predicted_data']]
        dic_best_model_sub[city] = df
        
    dic_best_model[var] = dic_best_model_sub
    
path = 'Data/SIR_Origin/*.csv'
dic_SIRs = Load_files.load_files(path, -17, -15)
data = dic_SIRs['서울']
dic_alpha = {}

for city in cities:
    df = dic_SIRs[city]
    dic_alpha[city] = df['alpha']

In [63]:
dic_best_model['up1']['강원']

,Date,Actual_data,Predicted_data
0,2020-08-16,81.0000,89.731540
1,2020-08-17,82.0000,89.763480
2,2020-08-18,84.0000,89.426476
3,2020-08-19,90.0000,93.304280
4,2020-08-20,95.0000,98.942350
...,...,...,...
140,2021-01-03,1301.0000,1246.998000
141,2021-01-04,1329.0000,1282.137200
142,2021-01-05,1356.0001,1308.846900
143,2021-01-06,1372.0000,1336.630500


In [64]:
df_predicted = dic_best_model['up1']['강원']
x = df_predicted['Actual_data'][-7:]
y = df_predicted['Predicted_data'][-7:]
z = x - y

In [65]:
for var in var_list:
    sum_statistics = 0
    print(var)
    print('')
    for city in cities:
        print(city)
        print('')
        df = dic_best_model[var][city]
        x = df['Actual_data'][-7:]
        y = df['Predicted_data'][-7:]
        mean_xy = np.mean(x)[0] - np.mean(y)[0]
        if var in ['up1', 'up2']:
            ttest = stats.ttest_rel(x, y, alternative = 'less')
            wil = wilcoxon(x, y, alternative = 'less')
        else:
            ttest = stats.ttest_rel(x, y, alternative = 'greater')
            wil = wilcoxon(x, y, alternative = 'greater')
        print(f'x - p_value : {shapiro(x)[1]:.4f}')
        print(f'y - p_value : {shapiro(y)[1]:.4f}')
        if (shapiro(x)[1] < 0.05) or (shapiro(y)[1] < 0.05):
            print(f'wilcoxon - mean : {mean_xy:.2f}, p_value : {wil[1][0]:.4f}')
        else:
            print(f't-test - mean : {mean_xy:.2f}, p_value : {ttest[1][0]:.4f}')
        print('')
        sum_statistics += mean_xy
    print(f'{sum_statistics:.2f}, {sum_statistics/17:.2f}')

up1

강원

x - p_value : 0.9423
y - p_value : 0.9650
t-test - mean : 44.62, p_value : 1.0000

경기

x - p_value : 0.9461
y - p_value : 0.8849
t-test - mean : 318.31, p_value : 0.9990

경남

x - p_value : 0.9339
y - p_value : 0.9237
t-test - mean : 57.41, p_value : 1.0000

경북

x - p_value : 0.8892
y - p_value : 0.8870
t-test - mean : 0.12, p_value : 0.5114

광주

x - p_value : 0.4408
y - p_value : 0.5825
t-test - mean : 48.24, p_value : 0.9941

대구

x - p_value : 0.7077
y - p_value : 0.8600
t-test - mean : 25.49, p_value : 0.9995

대전

x - p_value : 0.8506
y - p_value : 0.5553
t-test - mean : -0.62, p_value : 0.4351

부산

x - p_value : 0.8859
y - p_value : 0.9065
t-test - mean : 67.56, p_value : 1.0000

서울

x - p_value : 0.9471
y - p_value : 0.9971
t-test - mean : -108.26, p_value : 0.0575

세종

x - p_value : 0.4644
y - p_value : 0.9102
t-test - mean : -0.20, p_value : 0.3932

울산

x - p_value : 0.7506
y - p_value : 0.0111
wilcoxon - mean : 63.77, p_value : 1.0000

인천

x - p_value : 0.6972
y - p_val

In [54]:
for city1 in cities:
    for city2 in cities:
        if city1 == city2:
            continue
        
        x = list(dic_alpha[city1])
        y = list(dic_alpha[city2])
        ttest = stats.ttest_rel(x, y)
        if ttest[1] < 0.05:
            print(city1, city2)
            print(f't_test - statistics : {ttest[0]:.4f}, p_value : {ttest[1]:.4f}')

경기 전남
t_test - statistics : 1.9778, p_value : 0.0482
대전 전남
t_test - statistics : 2.0714, p_value : 0.0385
서울 전남
t_test - statistics : 2.6147, p_value : 0.0090
서울 전북
t_test - statistics : 2.2256, p_value : 0.0262
서울 제주
t_test - statistics : 2.3543, p_value : 0.0187
전남 경기
t_test - statistics : -1.9778, p_value : 0.0482
전남 대전
t_test - statistics : -2.0714, p_value : 0.0385
전남 서울
t_test - statistics : -2.6147, p_value : 0.0090
전북 서울
t_test - statistics : -2.2256, p_value : 0.0262
제주 서울
t_test - statistics : -2.3543, p_value : 0.0187
